In [1]:
from pathlib import Path

import numpy as np
import pandas as pd

from lightcurvedb import db
import pyticdb
from qlp.estools.bin.util import candidates_from_cli_args
from qlp.lctools.lightcurve import QLPLightCurve
from qlp.lctools.bls.bls_result import BLSResult
from qlp.reports.astronet.helpers import DBLCGetter
from qlp.util.util import Candidate

from astronet.direct_tensor.predict import batch_predict

2024-04-08 14:37:32.558022: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-08 14:37:32.599488: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:7630] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-08 14:37:32.599535: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-08 14:37:32.599546: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1500] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-08 14:37:32.609040: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-08 14:37:32.609752: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
sector = 75
cand_file = "/pdo/qlp-data/sector-75/ffi/run/pcs_cam1_single.ls"
cand_list = np.loadtxt(cand_file).astype(int)
candidates = candidates_from_cli_args("/pdo/qlp-data/sector-75/ffi", cand_list)
candidates

[Candidate(tic=53498154, sector_dir='/pdo/qlp-data/sector-75/ffi', cam=1, ccd=4, planetno=1),
 Candidate(tic=99869022, sector_dir='/pdo/qlp-data/sector-75/ffi', cam=1, ccd=2, planetno=1),
 Candidate(tic=147950620, sector_dir='/pdo/qlp-data/sector-75/ffi', cam=1, ccd=2, planetno=1),
 Candidate(tic=147951030, sector_dir='/pdo/qlp-data/sector-75/ffi', cam=1, ccd=2, planetno=1),
 Candidate(tic=148679712, sector_dir='/pdo/qlp-data/sector-75/ffi', cam=1, ccd=4, planetno=1),
 Candidate(tic=153949511, sector_dir='/pdo/qlp-data/sector-75/ffi', cam=1, ccd=2, planetno=1),
 Candidate(tic=252479260, sector_dir='/pdo/qlp-data/sector-75/ffi', cam=1, ccd=3, planetno=1),
 Candidate(tic=349827430, sector_dir='/pdo/qlp-data/sector-75/ffi', cam=1, ccd=2, planetno=1),
 Candidate(tic=416537241, sector_dir='/pdo/qlp-data/sector-75/ffi', cam=1, ccd=2, planetno=1),
 Candidate(tic=417931607, sector_dir='/pdo/qlp-data/sector-75/ffi', cam=1, ccd=1, planetno=1),
 Candidate(tic=522217469, sector_dir='/pdo/qlp-data/

In [3]:
get_lightcurve = DBLCGetter(candidates)

In [4]:
bls_columns = ["tic_id", "Epoc", "Per", "Depth", "Dur"]
bls_catalog = pd.DataFrame(
    [
        [result.tic_id, result.transit.transit_center, result.transit.period, result.transit.depth_ppm, result.transit.duration_days]
        for result in map(BLSResult.from_candidate, candidates)
    ],
    columns=bls_columns,
)
bls_catalog = bls_catalog.dropna().astype({"tic_id": "int64"})

tic_query_fields = ["id", "mass", "rad", "tmag"]
tic_columns = ["tic_id", "SMass", "SRad", "Tmag"]
query_result = pyticdb.query_by_id([c.tic for c in candidates], *tic_query_fields)
tic_catalog = pd.DataFrame(query_result, columns=tic_columns)

tce_catalog = tic_catalog.merge(bls_catalog, on="tic_id")
tce_catalog = tce_catalog[np.isfinite(tce_catalog["Per"])]
tce_catalog["SRadEst"] = np.nan
tce_catalog = tce_catalog.rename(columns={"tic_id": "TIC ID"})
tce_catalog["Astro ID"] = tce_catalog["TIC ID"]
tce_catalog

,TIC ID,SMass,SRad,Tmag,Epoc,Per,Depth,Dur,SRadEst,Astro ID
0,53498154,0.923,0.827372,10.03260,1913.706112,13.731523,2095.214817,0.165218,NaN,53498154
1,99869022,0.990,1.535520,10.42210,1686.217494,3.671594,4007.044925,0.137332,NaN,99869022
2,147950620,0.918,0.992671,10.47820,1684.933509,2.310616,6910.511792,0.076929,NaN,147950620
3,147951030,0.962,0.876335,10.45470,1691.790021,8.812679,786.823764,0.122769,NaN,147951030
4,148679712,0.946,1.060300,10.13450,1906.442232,4.924625,803.211558,0.145213,NaN,148679712
5,153949511,0.976,0.857648,9.76775,1684.958763,14.856047,1131.351605,0.177329,NaN,153949511
6,252479260,0.930,1.041810,9.00700,1871.638539,3.212199,14428.685986,0.124953,NaN,252479260
7,349827430,1.250,1.548670,7.91790,1684.859338,5.551238,9492.671459,0.235230,NaN,349827430
8,416537241,NaN,NaN,9.61180,1683.890767,0.617929,164.146992,0.075935,NaN,416537241
9,417931607,1.040,1.028520,8.99890,1694.321974,16.537649,607.663749,0.141518,NaN,417931607


In [5]:
checkpoints_dir = Path("/pdo/users/dmuth/mnt/tess/fa1t_38_run_1")

In [6]:
predictions = batch_predict(checkpoints_dir, tce_catalog, get_lightcurve, "triage", nprocs=5)
predictions

/pdo/users/haviland/.local/lib/python3.9/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/pdo/users/haviland/.local/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/pdo/users/haviland/.local/lib/python3.9/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/pdo/users/haviland/.local/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/pdo/users/haviland/.local/lib/python3.9/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/pdo/users/haviland/.local/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/pdo

11/11 [==============================] - 0s 5ms/step


,,disp_E,disp_N,disp_J,disp_S,disp_B
Astro ID,model_no,,,,,
53498154,0,0.874173,0.037733,1.302903e-01,0.003320,0.001531
99869022,0,0.999905,0.000065,1.015265e-05,0.000058,0.000071
147950620,0,0.999897,0.000045,7.149983e-06,0.000076,0.000071
147951030,0,0.998884,0.002849,6.145060e-04,0.000416,0.000851
148679712,0,0.998227,0.004288,1.084110e-03,0.000376,0.000411
...,...,...,...,...,...,...
252479260,9,0.999984,0.000008,9.734507e-07,0.000011,0.000006
349827430,9,0.999984,0.000009,9.511674e-07,0.000011,0.000008
416537241,9,0.017427,0.493446,8.519249e-01,0.004619,0.166036


In [7]:
predictions.groupby(level="Astro ID").max()

,disp_E,disp_N,disp_J,disp_S,disp_B
Astro ID,,,,,
53498154,0.985635,0.060139,0.548690,0.005598,0.001531
99869022,0.999943,0.000174,0.000014,0.000126,0.000071
147950620,0.999959,0.000104,0.000008,0.000127,0.000071
147951030,0.999012,0.012363,0.001534,0.002808,0.001645
148679712,0.998835,0.008310,0.002469,0.002284,0.000529
153949511,0.999649,0.001967,0.000625,0.000857,0.000293
252479260,0.999984,0.000043,0.000004,0.000049,0.000034
349827430,0.999984,0.000048,0.000003,0.000074,0.000032
416537241,0.069697,0.559191,0.888954,0.006425,0.166036
